In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import tensorflow as tf
import os
import numpy as np
import tensorflow_io as tfio

from helper_functions import (
    convert_prefetchdataset_to_numpy_arrays,
    lite_model_from_file_predicts_dataset,
    get_file_size, 
    convert_bytes,
    evaluate_prediction,
    full_int_model_predict
)


from sklearn.metrics import confusion_matrix

In [3]:
desired_length_of_audio = 48000
sample_rate = 16000

In [4]:
# Take all audio from testing dataset and create spectrograms from them
# We will use spectrograms for models testing
directory = 'dataset/testing'

x_data = []
y_data = []
for root, dirs, files in os.walk(directory):
    for file in files:
        full_file_name = os.path.join(root, file)

        if "non_target" in str(full_file_name):
            class_encoded = 0
        elif "target" in str(full_file_name):
            class_encoded = 1

        audio, sr = tf.audio.decode_wav(tf.io.read_file(full_file_name))
        audio = tf.squeeze(audio, axis=-1)
        resampled_audio = tfio.audio.resample(audio, rate_in=48000, rate_out=sample_rate)
        audio_length = tf.shape(resampled_audio)[0]
        if audio_length < desired_length_of_audio:
            resampled_audio = tf.pad(resampled_audio, [[0, desired_length_of_audio - audio_length]], mode='CONSTANT')
        else:
            resampled_audio = resampled_audio[:desired_length_of_audio]
        resampled_audio = tf.expand_dims(resampled_audio, axis=-1).numpy()

        x_data.append(resampled_audio)
        y_data.append(class_encoded)

# input data should be in numpy array, not in list
x_data_np = np.array(x_data)
y_data_np = np.array(y_data)

2024-06-20 11:36:16.241453: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 11:36:16.241886: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-20 11:36:16.345981: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-06-20 11:36:16.355858: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [5]:
for i in x_data_np:
    print(i.shape)
    break

(48000, 1)


### CNN

##### CNN initial model in keras format

In [7]:
cnn_initial_model_path = 'time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_baseline.keras'
convert_bytes(get_file_size(cnn_initial_model_path), "KB")

File size: 712.659 Kilobytes


In [8]:
cnn_initial_model = tf.keras.models.load_model(cnn_initial_model_path)
# input data should be in numpy array
y_pred_prob = cnn_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

2024-06-20 11:37:36.248835: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 267456000 exceeds 10% of free system memory.


Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%


##### CNN tf Lite model (without any additional quantization techniques)

In [66]:
cnn_tflite_model_path = 'time_series_models_from_notebooks/cnn/cnn_time_series_16kHz.tflite'
convert_bytes(get_file_size(cnn_tflite_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_tflite_model_path, x_data_np, y_data_np)

File size: 231.473 Kilobytes
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%


##### CNN tf Lite model + Post Training Dynamic range quantization

In [67]:
cnn_drq_model_path = 'time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_drq.tflite'
convert_bytes(get_file_size(cnn_drq_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_drq_model_path, x_data_np, y_data_np)

File size: 63.078 Kilobytes
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%


##### CNN tf Lite model + Float 16 quantization

In [68]:
cnn_float16q_model_path = 'time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_float16q.tflite'
convert_bytes(get_file_size(cnn_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_float16q_model_path, x_data_np, y_data_np)

File size: 119.863 Kilobytes
Accuracy: 96.12%
Recall: 92.14%
Precision: 95.91%
F1-score: 93.99%


##### CNN tf Lite model + Full integer quantization

In [10]:
cnn_full_int_q_model_path = 'time_series_models_from_notebooks/cnn/cnn_time_series_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(cnn_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(cnn_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 63.898 Kilobytes


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy: 83.35%
Recall: 51.53%
Precision: 95.93%
F1-score: 67.05%


### SqueezeNet

##### SqueezeNet initial model in keras format

In [12]:
squeezenet_initial_model_path = 'time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_baseline.keras'
convert_bytes(get_file_size(squeezenet_initial_model_path), "KB")

File size: 531.003 Kilobytes


In [13]:
squeezenet_initial_model = tf.keras.models.load_model(squeezenet_initial_model_path)
# input data should be in numpy array
y_pred_prob = squeezenet_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

2024-06-20 11:58:12.311205: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 267456000 exceeds 10% of free system memory.


Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%


##### SqueezeNet tf Lite model (without any additional quantization techniques)

In [73]:
squeezenet_tflite_model_path = 'time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz.tflite'
convert_bytes(get_file_size(squeezenet_tflite_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_model_path, x_data_np, y_data_np)

File size: 150.871 Kilobytes
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%


##### SqueezeNet tf Lite model + Post Training Dynamic range quantization

In [74]:
squeezenet_tflite_drq_model_path = 'time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_drq.tflite'
convert_bytes(get_file_size(squeezenet_tflite_drq_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_drq_model_path, x_data_np, y_data_np)

File size: 89.656 Kilobytes
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%


##### SqueezeNet tf Lite model + Float 16 quantization

In [75]:
squeezenet_float16q_model_path = 'time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float16q.tflite'
convert_bytes(get_file_size(squeezenet_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(squeezenet_float16q_model_path, x_data_np, y_data_np)

File size: 97.117 Kilobytes
Accuracy: 86.22%
Recall: 96.07%
Precision: 71.66%
F1-score: 82.09%


##### CNN tf Lite model + Full integer quantization

In [11]:
squeezenet_full_int_q_model_path = 'time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(squeezenet_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(squeezenet_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 87.039 Kilobytes
Accuracy: 70.78%
Recall: 68.12%
Precision: 54.45%
F1-score: 60.52%
